# Predicting Customer Churn - Sci-kit Learn

This notebook uses Python and SKLearn to build a model to predict customer churn based upon 3 datasets
 - Customer Demographics
 - Customer Transactions
 - Customer Churn Flag

Whilst it closely follows the steps within the Modeler Flows section of the lab, it is not an exact replication

## Step 1: Read Individual Data Files in to Pandas Dataframes

#### Note about reading data files into Notebooks in Watson Studio Desktop
Unlike Jupyter notebooks in Watson Studio Cloud and Local, there is no "Insert to code" option within Watson Studio Desktop. Therefore code has to be written by the user to access data files.<br>
- The sample code below shows how to read the data sets from the project.<br>
- Documentation on how to read data in to notebooks can be found at [https://www.ibm.com/support/knowledgecenter/SSBFT6_1.1.0/wsj/analyze-data/load-and-access-data.html](https://www.ibm.com/support/knowledgecenter/SSBFT6_1.1.0/wsj/analyze-data/load-and-access-data.html).<br>

In [ ]:
# Load the libraries required to read the data from the WSD project assets

import pandas as pd
from project_lib import Project
project = Project()

In [ ]:
# Obtain a list of file names stored within the project assets

filename = project.get_files()
print(filename)

In [ ]:
# Display the index number for each file in the project data assets

index = 1
for filename in project.get_files():
    print ("File " + str(index) + " - " + str(filename))
    index = index + 1

In [ ]:
# Read the demographic data from the project assets

demographic = pd.read_csv(project.get_file('Demographic Data.csv'))
demographic.head()

In [ ]:
# Read the transaction data from the project assets

transaction = pd.read_csv(project.get_file('Transaction Data.csv'))
transaction.head()

In [ ]:
# Read the churn flag from the project assets

churn = pd.read_csv(project.get_file('Churn Data.csv'))
churn.head()

In [ ]:
# Validate that we have 3 dataframes, and each data frame has different fields within it

print("The Demographic dataframe should have 8 columns within it.")
print("Column read are:",list(demographic))
print()
print("The Transaction dataframe should have 8 columns within it.")
print("Column read are:",list(transaction))
print()
print("The Churn dataframeshould have 2 columns within it.")
print("Column read are:",list(churn))

## Step 2: Join the Dataframes and check the Data Types

In [ ]:
# Join the 3 files together in to a single dataframe called "merged"

merged = pd.merge(pd.merge(demographic, transaction, on='ID'),churn,on='ID')
merged.head(3)

In [ ]:
# Check the data types

merged.dtypes

## Step 3: Explore the data

In [ ]:
print ("There are " + str(merged.shape[0]) + " records and " + str(merged.shape[1]) + " fields in the dataset.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set(style="darkgrid")

In [ ]:
g1 = sns.countplot(data=merged, x='Churn', order=merged.Churn.value_counts().index)
plt.title('Churn Rates')
plt.ylabel('Count of Churn')
plt.ylim(0, 1600)
#Add percentages to the graph
total = float(len(merged)) #one person per row
for p in g1.patches:
    height = p.get_height()
    g1.text(p.get_x()+p.get_width()/2.,
            height + 1,
            '{0:.0%}'.format(height/total),
            ha="center") 
plt.show()


In [ ]:
sns.catplot(x="Churn", y="Avg_Spend_Per_Store_Visit",
                 hue="Status", col="Paymethod",
                 data=merged, kind="bar",
                 height=7, aspect=.81,capsize=.05);

In [ ]:
sns.catplot(x="Churn", y="Trips_To_The_Website_Yr",
                 hue="Status", col="Paymethod",
                 data=merged, kind="bar",
                 height=7, aspect=.81,capsize=.05);

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(15,8))
g = sns.lineplot(x="Number_Of_Transactions_Current_Year", y="Avg_Spend_Per_Store_Visit", hue="Customer_Loyality_Code",data=merged, linewidth=2.5)


## Step 4: Build the Predictive Model - Random Forest from Sci-kit Learn

In [ ]:
# Only need to run this once within your environment

!pip install sklearn-pandas

In [ ]:
# Import the required scikit-learn libraries

import sklearn
import warnings
import numpy as np
import urllib3, requests, json

from scipy.stats import chi2_contingency,ttest_ind
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 999

In [ ]:
# Convert the VarChar columns to numeric columns

le = LabelEncoder()
merged.loc[:,'Churn']= le.fit_transform(merged.loc[:,'Churn'])
merged.head()

In [ ]:
# Check the values for Churn

merged.groupby(['Churn']).size()

In [ ]:
# define the label (Target Field) and features

y = np.float32(merged.Churn)
x = merged.drop(['Churn','ID'], axis = 1)

In [ ]:
# List the fields used as inputs (features) to predict churn with

list(x)

In [ ]:
# Apply the LabelEncoder to encode the input features in numeric form where applicable

from sklearn_pandas import DataFrameMapper
mapper = DataFrameMapper([
     ('Sex', LabelEncoder()),
     ('Region', LabelEncoder()),
     ('Children',None),
     ('Est_Income',None),
     ('AgeGroup', LabelEncoder()),    
     ('Car_Owner', LabelEncoder()),
     ('Status', LabelEncoder()),
     ('Paymethod', LabelEncoder()),
     ('LocalBilltype', LabelEncoder()),
     ('Customer_Segments', LabelEncoder()),
     ('Trips_To_The_Website_Yr',None),
     ('Avg_Spend_Per_Store_Visit',None),
     ('Customer_Loyality_Code',LabelEncoder()),
     ('Number_Of_Transactions_Current_Year',None)
    ])

In [ ]:
# split the data to training and testing set

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# fit the model

import sklearn.pipeline
from sklearn.preprocessing import OneHotEncoder

random_forest = RandomForestClassifier()
steps = [('mapper', mapper),('RandonForestClassifier', random_forest)]
pipeline = sklearn.pipeline.Pipeline(steps)
model=pipeline.fit( x_train, y_train )
model

## Step 5: Evaluate The Model

In [ ]:
# call pipeline.predict() on your x_test data to make a set of test predictions which are written to series y_prediction

y_prediction = pipeline.predict(x_test)
y_probability = pipeline.predict_proba(x_test)

# Evaluate the model using sklearn.classification_report()
report = sklearn.metrics.classification_report(y_test, y_prediction )
accuracy = sklearn.metrics.accuracy_score(y_test, y_prediction )
print(report)
print("Overall Model Accuracy: " + str(accuracy))

In [ ]:
# Reset the index on the x_train data so that the join will match record by record and not require a key
x_test.reset_index(drop=True, inplace=True)

#Write the Actual and Predicted Mortgage Default values in to dataframes 
y_test_df = pd.DataFrame(y_test,columns=['Churn'])
y_pred_df = pd.DataFrame(y_prediction,columns=['Pred Churn'])
y_prob_df = pd.DataFrame(y_probability,columns=['Prob Non-Churn','Prob Churn'])

# Combine the three dataframes by index value rather than key field
scored_df = pd.concat([x_test, y_test_df, y_pred_df, y_prob_df], axis=1)
scored_df.head()


In [ ]:
# Display a table showing the feature importance for each of the input fields

feature_imp = pd.DataFrame(random_forest.feature_importances_, index = x_test.columns,columns=['Importance']).sort_values('Importance',ascending=False)
feature_imp['Feature'] = feature_imp.index
feature_imp.reset_index(drop=True)
feature_imp.head(15)

In [ ]:
# Display the feature importance as a visualisation

sns.factorplot(y="Feature",x="Importance", data=feature_imp,kind="bar",palette="Blues",size=6,aspect=2)
plt.title('Mortgage Default - Feature Importance')

## Step 6: Save the Model to WML Server

#### <font color=red>Note: This code was written without access to a working WML Server, therefore requires validation but is based upon published documentation!<font>

In [ ]:
!pip install --upgrade watson-machine-learning-client-V4

from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# Input the WML Server credentials that were shared by your lab instructor

wml_credentials = {
     "url": "https://****",
     "username": "****",
     "password": "****",
     "instance_id": "wml_server"
 }

In [ ]:
#Save the credentials to the variable clients:

client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# Obtain the UId of your space

def guid_from_space_name(client, space_name):
     instance_details = client.service_instance.get_details()
     space = client.spaces.get_details()
     return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

<b>Action:</b> Enter the name of your deployment space in the code below: space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE').

In [ ]:
# Enter the name of your deployment space here:

space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

In [ ]:
# Model Metadata.

meta_props={
    client.repository.ModelMetaNames.NAME: "Churn Prediction Model - SKLeanr",
    client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3.6",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20",
    client.repository.ModelMetaNames.SPACE_UID: space_uid
}

Save the churn model, along with the associated meta-data

In [ ]:
# Create the model artifact.
model_artifact = client.repository.store_model(mod,
                                                   meta_props=meta_props,
                                                   training_data=X_train,
                                                   training_target=y_train)

## Step 7: Deploy the Model

In [ ]:
As with Watson Studio and Watson Machine Learning within Cloud Pak for Data, it is possible to create the model deployments from within the notebook itself. <br>
In this lab, the above code has saved the model to the WML Server and the model can be deployed manually via the GUI